# Getting images using the new SIAv2 application

In [ ]:
import os
import random
import requests
import matplotlib.pyplot as plt

import pyvo
from pyvo.dal import SIA2Service
from astropy.coordinates import SkyCoord
from astropy.io import fits
from astropy import units as u
from astropy.time import Time
from astropy.utils.data import download_file
from astropy.visualization import imshow_norm, ZScaleInterval
from astropy.visualization.stretch import  SqrtStretch

from lsst.rsp import get_datalink_result
from lsst.rsp.utils import get_pyvo_auth, get_access_token, get_service_url
from lsst.rsp.service import get_siav2_service

In [ ]:
# Specify the SIA service to use
service = get_siav2_service("dp02")

In [ ]:
# Setup auth for datalinker
import requests
s = requests.Session()
tok = get_access_token()
s.headers["Authorization"] = f"Bearer {tok}"
auth = pyvo.auth.authsession.AuthSession()
auth.credentials.set("lsst-token", s)
auth.add_security_method_for_url(get_service_url("datalink"), "lsst-token")

In [ ]:
# Execute a query looking for pictures within a circle
# and show them as a table
from astropy import units as u

coords = SkyCoord(55.777353, -32.544474, unit=(u.deg, u.deg))

t1 = Time("60550.31803461111", format='mjd')
t2 = Time("60550.31838182871", format='mjd')

# Create Time objects
results = service.search(
    pos=(coords, 0.10*u.deg),
    time=[t1, t2]
)

In [ ]:
print(results)

In [ ]:
# Extract the access URL from the result in the first row
dataLinkUrl = random.choice(results).access_url

In [ ]:
print(dataLinkUrl)

In [ ]:
from pyvo.dal.adhoc import DatalinkResults
dl_result = DatalinkResults.from_result_url(
    dataLinkUrl, session=auth
)
print(dl_result)

In [ ]:
# Retrieve the datalink votable document and
# show it in the notebook
dl_result

In [ ]:
# Full image of calexp - not a cutout
image_url = dl_result.getrecord(0).get('access_url')
image_url

In [ ]:
# Now let's download the image 
filename = download_file(image_url)
filename

In [ ]:
hdulist = fits.open(image_url)

for hdu in hdulist:
    print(hdu.name)

In [ ]:
# Let's plot the image and see what it looks like...
image = hdulist[1].data

fig = plt.figure()
ax = fig.add_subplot(1, 1, 1)
im = imshow_norm(image, ax, origin='lower', interval=ZScaleInterval(), stretch=SqrtStretch(), cmap='gray')
fig.colorbar(im[0])